In [32]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
from dwave.samplers import SimulatedAnnealingSampler, TabuSampler
import dimod
import psutil
import time
from scipy.sparse import lil_matrix, csc_matrix, block_diag
from dwave.system import DWaveSampler, EmbeddingComposite
import tracemalloc




In [33]:
N_MODULES = 3
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX] * N_MODULES,
    ly=[LY] * N_MODULES,
    z=[i + Z_SPACING for i in range(N_MODULES)]
)

generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi / 6
)

N_PARTICLES = 2
event = generator.generate_event(N_PARTICLES)
event.hits

[Hit(hit_id=0, x=-0.06320613921182602, y=-0.012856168016378657, z=1.0, module_id=0, track_id=0),
 Hit(hit_id=3, x=-0.2718702908710253, y=-0.09837095607652353, z=1.0, module_id=0, track_id=1),
 Hit(hit_id=1, x=-0.12641227842365205, y=-0.025712336032757313, z=2.0, module_id=1, track_id=0),
 Hit(hit_id=4, x=-0.5437405817420506, y=-0.19674191215304707, z=2.0, module_id=1, track_id=1),
 Hit(hit_id=2, x=-0.18961841763547807, y=-0.03856850404913597, z=3.0, module_id=2, track_id=0),
 Hit(hit_id=5, x=-0.8156108726130761, y=-0.29511286822957056, z=3.0, module_id=2, track_id=1)]

In [34]:
params = {
    'alpha': 1.0,
    'beta': 1.0,
    'lambda': 100.0,} 

def generate_hamiltonian_optimized(event, params):
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    modules = sorted(event.modules, key=lambda module: module.z)

    
    segments = [
        em.segment(from_hit, to_hit)
        for idx in range(len(modules) - 1)
        for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)
    ]

    N = len(segments)  

    #initialize sparse block matrices for effviciency
    A_ang_blocks = []
    A_bif_blocks = []
    A_inh_blocks = []
    b = np.zeros(N)

    #total Hamiltonian into smaller blocks->> better for memory
    block_size = 500  
    num_blocks = (N + block_size - 1) // block_size  

    for block_idx in range(num_blocks):
        start_idx = block_idx * block_size
        end_idx = min(start_idx + block_size, N)

        #lil_matrix for each block
        A_ang_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_bif_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_inh_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)

        #filling of papricas
        for i in range(start_idx, end_idx):
            seg_i = segments[i]
            vect_i = seg_i.to_vect()
            norm_i = np.linalg.norm(vect_i)

            for j in range(i + 1, end_idx):  #
                seg_j = segments[j]
                vect_j = seg_j.to_vect()
                norm_j = np.linalg.norm(vect_j)

                cosine = np.dot(vect_i, vect_j) / (norm_i * norm_j)
                if np.abs(cosine - 1) < 1e-9:
                    A_ang_block[i - start_idx, j - start_idx] = 1
                    A_ang_block[j - start_idx, i - start_idx] = 1  # Symmetry with positive sign

                if seg_i.from_hit == seg_j.from_hit and seg_i.to_hit != seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                if seg_i.from_hit != seg_j.from_hit and seg_i.to_hit == seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                s_ab = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)
                if s_ab > 0:
                    A_inh_block[i - start_idx, j - start_idx] = beta * s_ab * s_ab
                    A_inh_block[j - start_idx, i - start_idx] = beta * s_ab * s_ab  # Symmetry with positive sign

        A_ang_blocks.append(A_ang_block)
        A_bif_blocks.append(A_bif_block)
        A_inh_blocks.append(A_inh_block)

    # combine withblock diagonal
    A_ang = block_diag(A_ang_blocks, format='csc')
    A_bif = block_diag(A_bif_blocks, format='csc')
    A_inh = block_diag(A_inh_blocks, format='csc')

    A = -1 * (A_ang + A_bif + A_inh)

    return A, b, segments

#performance measurement
process = psutil.Process()

start_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
start_time = time.time()

A, b, segments = generate_hamiltonian_optimized(event, params)
end_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
end_time = time.time()

memory_used = end_memory - start_memory
time_taken = end_time - start_time
print(A.toarray())
print(f"Memory used: {memory_used:.2f} MB")
print(f"Time taken: {time_taken:.6f} seconds")

[[ 0.  1.  1.  0. -1.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0. -1.]
 [-1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  0.  0. -1.  0.  1.  1.  0.]]
Memory used: 0.00 MB
Time taken: 0.004986 seconds


In [35]:
import dimod
import psutil
import time
from scipy.sparse import csc_matrix

def qubosolver(A, b):


    #Keep A sparse
    A = csc_matrix(A)

    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)

    #vectors for efficiency 
    bqm.add_variables_from({i: b[i] for i in range(len(b))})

    row, col = A.nonzero()  
    for i, j in zip(row, col):
        if i != j:  
            bqm.add_interaction(i, j, A[i, j])

    sampler = SimulatedAnnealingSampler()
    response = sampler.sample(bqm, num_reads=100)

    best_sample = response.first.sample
    sol_sample = np.fromiter(best_sample.values(), dtype=int)  


    print(f"Solution:{sol_sample}")


    return sol_sample

qubosolver(A, b)


Solution:[1 0 0 1 1 0 0 1]


array([1, 0, 0, 1, 1, 0, 0, 1])

In [36]:
import dimod
def qubosolverTABU(A, b):
    # Keep A sparse
    A = csc_matrix(A)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)

    # Vectors for efficiency 
    bqm.add_variables_from({i: b[i] for i in range(len(b))})

    row, col = A.nonzero()  
    for i, j in zip(row, col):
        if i != j:  
            bqm.add_interaction(i, j, A[i, j])

    sampler = TabuSampler()
    response = sampler.sample(bqm, num_reads=50)

    best_sample = response.first.sample
    sol_sample = np.fromiter(best_sample.values(), dtype=int)  



    return sol_sample
qubosolverTABU(A, b)

array([1, 0, 0, 1, 1, 0, 0, 1])

In [5]:
def qubosolverQA(A, b):
    # Start tracing memory allocations
    tracemalloc.start()
    start_time = time.time()

    # Keep A sparse
    A = csc_matrix(A)

    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    bqm.add_variables_from({i: b[i] for i in range(len(b))})

    row, col = A.nonzero()
    for i, j in zip(row, col):
        if i != j:
            bqm.add_interaction(i, j, A[i, j])

    # Use D-Wave's quantum sampler
    sampler = EmbeddingComposite(DWaveSampler(token='DEV-2df6769d96a5ddaec4b4e037d3a7a92833582e9c'))
    response = sampler.sample(bqm, num_reads=100)

    best_sample = response.first.sample
    sol_sample = np.fromiter(best_sample.values(), dtype=int)

    end_time = time.time()
    # Get the current and peak memory usage
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    memory_used = peak / (1024 ** 2)
    time_taken = end_time - start_time

    print(f"Solution: {sol_sample}")
    print(f"Peak Memory Used: {memory_used:.2f} MB")
    print(f"Time Taken: {time_taken:.6f} seconds")

    return sol_sample, memory_used, time_taken

qubosolverQA(A, b)

Solution: [1 0 0 1 1 0 0 1]
Peak Memory Used: 50.20 MB
Time Taken: 12.614265 seconds


(array([1, 0, 0, 1, 1, 0, 0, 1]), 50.19823360443115, 12.614264726638794)

In [6]:
from dwave.system import LeapHybridSampler
import os

os.environ['DWAVE_API_TOKEN'] = 'DEV-2df6769d96a5ddaec4b4e037d3a7a92833582e9c'
import tracemalloc
import time
import numpy as np
from scipy.sparse import csc_matrix
from dimod import BinaryQuadraticModel

# Rewritten classical QUBO solver to use the Leap Hybrid Solver
def qubosolverHr(A, b):
    # Start memory and time tracking with tracemalloc and time
    tracemalloc.start()
    start_time = time.time()

    # Keep A sparse
    A = csc_matrix(A)

    # Define a Binary Quadratic Model (BQM) for the QUBO problem
    bqm = BinaryQuadraticModel.empty(dimod.BINARY)

    # Add the linear terms from vector b
    bqm.add_variables_from({i: b[i] for i in range(len(b))})

    # Add the quadratic terms (interaction terms) from matrix A
    row, col = A.nonzero()  # Get non-zero entries in the matrix A
    for i, j in zip(row, col):
        if i != j:  # Only consider off-diagonal terms (interactions)
            bqm.add_interaction(i, j, A[i, j])

    # Use D-Wave's Leap Hybrid Solver, which handles large-scale problems
    sampler = LeapHybridSampler()

    # Solve the QUBO problem using the hybrid solver
    response = sampler.sample(bqm)

    # Extract the best sample (solution) from the response
    best_sample = response.first.sample
    sol_sample = np.fromiter(best_sample.values(), dtype=int)

    # Stop memory tracking with tracemalloc and get memory usage details
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # Convert memory from bytes to megabytes
    memory_used_mb = current / (1024 ** 2)
    peak_memory_used_mb = peak / (1024 ** 2)

    # Stop time tracking
    end_time = time.time()

    # Calculate time taken
    time_taken = end_time - start_time

    # Output solution and performance details
    print(f"Solution: {sol_sample}")
    print(f"Memory used: {memory_used_mb:.2f} MB (Peak: {peak_memory_used_mb:.2f} MB)")
    print(f"Time: {time_taken:.6f} seconds")

    return sol_sample, time_taken, peak_memory_used_mb
qubosolverHr(A, b)

KeyboardInterrupt: 

In [23]:
from itertools import product
from scipy.sparse import csc_matrix

def brute_force(A, b):
    A = csc_matrix(A)
    n = len(b) 

    best_solution = None
    best_energy = np.inf
    for config in product([0, 1], repeat=n):
        config = np.array(config)
        
        #calculate energy: E = x^T A x + b^T x
        energy = config @ A @ config + b @ config
        if energy < best_energy:
            best_energy = energy
            best_solution = config

    print(f"Best Solution: {best_solution}")
    print(f"Best Energy: {best_energy}")

    return best_solution


brute_force_solution = brute_force(A, b)
print(f"Brute Force Solution: {brute_force_solution}")




Best Solution: [1 0 0 1 1 0 0 1]
Best Energy: -4.0
Brute Force Solution: [1 0 0 1 1 0 0 1]
